# Phenotype Synergy Analysis

This notebook contains code to interprete results from the synergy score analysis. 

In [1]:
import numpy as np
import pandas as pd
import datetime
import math
import os
import sys
import logging
mf_module_path = os.path.abspath(os.path.join('../python'))
if mf_module_path not in sys.path:
    sys.path.append(mf_module_path)
import mf
import mf_random
import ontology
import pickle

In [2]:
hpo = ontology.Ontology('/Users/zhangx/git/human-phenotype-ontology/hp.obo')
hpo_term_map = hpo.term_id_2_label_map()

# Mutual information without considering diagnosis

This section analyzes the mutual information between phenotype pairs (labHpo-labHpo, textHpo-labHpo, textHpo-textHpo) in regardless of diagnosis. 

Note that the same information is also produced as a side product when we calculate mutual information in respect to a disease. Because we run simulations in the latter case, we additionally get their p values for the observed value. 

In [12]:
with open('../../../data/mf_regardless_of_diseases/summary_textHpo_labHpo.obj', 'rb') as f:
    summary_textHpo_labHpo = pickle.load(f)
with open('../../../data/mf_regardless_of_diseases/summary_textHpo_textHpo.obj', 'rb') as f:
    summary_textHpo_textHpo = pickle.load(f)  
with open('../../../data/mf_regardless_of_diseases/summary_labHpo_labHpo.obj', 'rb') as f:
    summary_labHpo_labHpo = pickle.load(f)

In [4]:
mf_textHpo_labHpo = mf.MutualInfoXY(summary_textHpo_labHpo)
mf_textHpo_textHpo = mf.MutualInfoXY(summary_textHpo_textHpo)
mf_labHpo_labHpo = mf.MutualInfoXY(summary_labHpo_labHpo)

For each type of summary statistics, we can map them into a MutualInfoXY instance. From there, we can get the mutual information matrix (or dataframe), and do necessary filtering. 

**Filter 1:**

If P1 and P2 are both from labHpo or textHpo, then remove rows where P1 is identical to P2

**Filter 2:**

If P1 and P2 are both from labHpo or textHpo, then a row for (a, b) is identical to (b, a). So one row is removed. 

**Filter 3:**

Because we automatically added ancestors terms if a child term is observed, P1 and P2 are expected to have high mutual information if they have dependency in HPO hierarchy. Such pairs are removed. 

### TextHpo -- LabHpo

In [41]:
df_mf_textHpo_labHpo = mf_textHpo_labHpo.mf_labeled()
# add labels
df_mf_textHpo_labHpo['P1_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_textHpo_labHpo.P1])
df_mf_textHpo_labHpo['P2_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_textHpo_labHpo.P2])

# filter rows: because P1 is from radiology reports and P2 is from lab tests, every pair is unique!
# we just remove dependent terms 
mask = np.array([df_mf_textHpo_labHpo.P1[i] != df_mf_textHpo_labHpo.P2[i] and
                 (hpo.exists_path(df_mf_textHpo_labHpo.P1[i], df_mf_textHpo_labHpo.P2[i]) or 
                 hpo.exists_path(df_mf_textHpo_labHpo.P2[i], df_mf_textHpo_labHpo.P1[i])) 
                 for i in np.arange(df_mf_textHpo_labHpo.shape[0])])

df_mf_textHpo_labHpo = df_mf_textHpo_labHpo.loc[np.logical_not(mask), :]
df_mf_textHpo_labHpo = df_mf_textHpo_labHpo.sort_values(by='mf', ascending=False).reset_index(drop=True)
df_mf_textHpo_labHpo.to_csv('../../../data/mf_regardless_of_diseases/mf_textHpo_labHpo.csv')
df_mf_textHpo_labHpo.head(n=5)

,P1,P2,mf,P1_label,P2_label
0,HP:0002202,HP:0020062,0.125660,Pleural effusion,Decreased hemoglobin concentration
1,HP:0002202,HP:0020061,0.121226,Pleural effusion,Abnormal hemoglobin concentration
2,HP:0002202,HP:0011014,0.121010,Pleural effusion,Abnormal glucose homeostasis
3,HP:0002202,HP:0011015,0.121010,Pleural effusion,Abnormal blood glucose concentration
4,HP:0002086,HP:0001939,0.120846,Abnormality of the respiratory system,Abnormality of metabolism/homeostasis


### TextHpo -- TextHpo

In [42]:
df_mf_textHpo_textHpo = mf_textHpo_textHpo.mf_labeled()
# add labels
df_mf_textHpo_textHpo['P1_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_textHpo_textHpo.P1])
df_mf_textHpo_textHpo['P2_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_textHpo_textHpo.P2])
# add p values if they exist

# apply filter1 and filter2
df_mf_textHpo_textHpo = df_mf_textHpo_textHpo.loc[df_mf_textHpo_textHpo.P1 < df_mf_textHpo_textHpo.P2, :].reset_index(drop=True)

# remove directly dependent terms 
mask = np.array([hpo.exists_path(df_mf_textHpo_textHpo.P1[i], df_mf_textHpo_textHpo.P2[i]) or
                 hpo.exists_path(df_mf_textHpo_textHpo.P2[i], df_mf_textHpo_textHpo.P1[i])
                 for i in np.arange(df_mf_textHpo_textHpo.shape[0])])
df_mf_textHpo_textHpo = df_mf_textHpo_textHpo.loc[np.logical_not(mask), :]
df_mf_textHpo_textHpo = df_mf_textHpo_textHpo.sort_values(by='mf', ascending=False).reset_index(drop=True)
df_mf_textHpo_textHpo.to_csv('../../../data/mf_regardless_of_diseases/mf_textHpo_textHpo.csv')
df_mf_textHpo_textHpo.head(n=5)

,P1,P2,mf,P1_label,P2_label
0,HP:0001892,HP:0011028,0.706285,Abnormal bleeding,Abnormality of blood circulation
1,HP:0011947,HP:0012649,0.625522,Respiratory tract infection,Increased inflammatory response
2,HP:0011947,HP:0012647,0.625481,Respiratory tract infection,Abnormal inflammatory response
3,HP:0010978,HP:0011947,0.565861,Abnormality of immune system physiology,Respiratory tract infection
4,HP:0011024,HP:0025033,0.543392,Abnormality of the gastrointestinal tract,Abnormality of digestive system morphology


### LabHpo -- LabHpo

In [43]:
df_mf_labHpo_labHpo = mf_labHpo_labHpo.mf_labeled()
# add labels
df_mf_labHpo_labHpo['P1_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_labHpo_labHpo.P1])
df_mf_labHpo_labHpo['P2_label'] = np.array([hpo_term_map.get(termid) for termid in df_mf_labHpo_labHpo.P2])

# apply filter1 and filter2
df_mf_labHpo_labHpo = df_mf_labHpo_labHpo.loc[df_mf_labHpo_labHpo.P1 < df_mf_labHpo_labHpo.P2, :].reset_index(drop=True)

# remove directly dependent terms 
mask = np.array([hpo.exists_path(df_mf_labHpo_labHpo.P1[i], df_mf_labHpo_labHpo.P2[i]) or 
                 hpo.exists_path(df_mf_labHpo_labHpo.P2[i], df_mf_labHpo_labHpo.P1[i]) 
                 for i in np.arange(df_mf_labHpo_labHpo.shape[0])])
df_mf_labHpo_labHpo = df_mf_labHpo_labHpo.loc[np.logical_not(mask), :]
df_mf_labHpo_labHpo = df_mf_labHpo_labHpo.sort_values(by='mf', ascending=False).reset_index(drop=True)
df_mf_labHpo_labHpo.to_csv('../../../data/mf_regardless_of_diseases/mf_labHpo_labHpo.csv')
df_mf_labHpo_labHpo.head(n=5)

,P1,P2,mf,P1_label,P2_label
0,HP:0002157,HP:0031970,0.744493,Azotemia,Abnormal blood urea nitrogen concentration
1,HP:0020061,HP:0031850,0.575781,Abnormal hemoglobin concentration,Abnormal hematocrit
2,HP:0020062,HP:0031851,0.546701,Decreased hemoglobin concentration,Reduced hematocrit
3,HP:0020058,HP:0020061,0.543036,Abnormal red blood cell count,Abnormal hemoglobin concentration
4,HP:0020058,HP:0031850,0.533428,Abnormal red blood cell count,Abnormal hematocrit


## Mutual information between textHpo and labHpo in respect to diagnoses

At each admission, patients could receive multiple diagnosis codes. One of them is designated as "primary" (in MIMIC, it has a rank of 1) and others secondary (rank 2, 3...). Therefore, the analysis was run under two scenerios: 
1. Only primary diagnosis is considered. 
2. All diagnoses are considered equally. 

Under the first scenerio, a patient is considered to be a case only if the corresponding billing code is listed as "primary". While in the second case, a patient is considered to be a case when the corresponding billing code is listed as primary or secondary.   

In [45]:
def mf_dataframes(mf_diagnosis_phenotypes, **p_values):
    """
    @param p_values: output from simulation
    """
    assert isinstance(mf_diagnosis_phenotypes, mf.MutualInfoXYz)
    # unpack p values
    p_mf_Xz = p_values.get('mf_Xz')
    p_mf_Yz = p_values.get('mf_Yz')
    p_mf_XY_z = p_values.get('mf_XY_z')
    p_mf_XY_given_z = p_values.get('mf_XY_given_z')
    p_synergy = p_values.get('synergy')
    p_mf_XY_omit_z = p_values.get('mf_XY_omit_z')
    
    X_labels, Y_labels = mf_diagnosis_phenotypes.vars_labels.values()
    M1 = len(X_labels)
    M2 = len(Y_labels)

    mf_Xz = mf_diagnosis_phenotypes.mutual_info_Xz()
    mf_Yz = mf_diagnosis_phenotypes.mutual_info_Yz()

    # mutual information between single phenotypes and diagnosis
    df_mf_Xz = pd.DataFrame(data={'X': X_labels, 'mf_Xz': mf_Xz})
    df_mf_Yz = pd.DataFrame(data={'Y': Y_labels, 'mf_Yz': mf_Yz})
    # add p values
    df_mf_Xz['p_mf_Xz'] = p_mf_Xz if p_mf_Xz is not None else np.repeat(-1, M1)
    df_mf_Yz['p_mf_Yz'] = p_mf_Yz if p_mf_Yz is not None else np.repeat(-1, M2)
    
    # joint and conditional mutual information, and synergy
    mf_XY_z = mf_diagnosis_phenotypes.mutual_info_XY_z()
    mf_XY_given_z = mf_diagnosis_phenotypes.mutual_info_XY_given_z()
    mf_synergy = mf_diagnosis_phenotypes.synergy_XY2z()
    
    # mutual information between phenotypes without considering diagnosis
    mf_XY_omit_z = mf_diagnosis_phenotypes.mutual_info_XY_omit_z()
    
    # mutual information between phenotype pairs and diagnosis
    df_mf_XY_z = pd.DataFrame()
    df_mf_XY_z['X'] = np.repeat(X_labels, M2)
    df_mf_XY_z['Y'] = np.tile(Y_labels, [M1])
    df_mf_XY_z['mf_Xz'] = np.repeat(mf_Xz, M2)
    df_mf_XY_z['mf_Yz'] = np.tile(mf_Yz, [M1])
    df_mf_XY_z['mf_XY_z'] = mf_XY_z.flat
    df_mf_XY_z['synergy'] = mf_synergy.flat   # synergy = mf_XY_z - mf_Xz - mf_Yz
    
    # mutual information between phenotypes after omiting diagnosis or conditioned on diagnosis
    df_mf_XY_z['mf_XY_omit_z'] = mf_XY_omit_z.flat
    df_mf_XY_z['mf_XY_given_z'] = mf_XY_given_z.flat
    
    # ratio of conditional mutual information / mutual information without considering diagnosis
    # synergy is also equal to mf_XY_given_z - mf_XY_omit_z
    # here we use ratio, which can be considered as an alternative of the above definition of synergy
    df_mf_XY_z['mf_ratio'] = df_mf_XY_z['mf_XY_given_z'] / df_mf_XY_z['mf_XY_omit_z']
    
    # add p values; otherwise, assign -1
    df_mf_XY_z['p_mf_Xz'] = np.repeat(p_mf_Xz, M2) if p_mf_Xz is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_Yz'] = np.tile(p_mf_Yz, [M1]) if p_mf_Yz is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_z'] = p_mf_XY_z.flat if p_mf_XY_z is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_synergy'] = p_synergy.flat if p_synergy is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_omit_z'] = p_mf_XY_omit_z.flat if p_mf_XY_omit_z is not None else np.repeat(-1, M1*M2)
    df_mf_XY_z['p_mf_XY_given_z'] = p_mf_XY_given_z.flat if p_mf_XY_given_z is not None else np.repeat(-1, M1*M2)
    
    
    # add raw counts: 8 additional columns
    joint_dist_keys = ['+++', '++-', '+-+', '+--', '-++', '-+-', '--+', '---']
    joint_dist_values = mf_diagnosis_phenotypes.m2.reshape([-1, 8]).astype(int)
    raw_counts = {joint_dist_keys[i]: joint_dist_values[:,i] for i in np.arange(8)}
    df_mf_XY_z = df_mf_XY_z.assign(**raw_counts)
    df_mf_XY_z['sum'] = np.sum(joint_dist_values, axis=-1) # it's a constant for all rows
    

    return df_mf_Xz, df_mf_Yz, df_mf_XY_z
    

def filter_df(df_mf_Xz, df_mf_Yz, df_mf_XY_z):
    df_merged = df_mf_XY_z

    ## filter out identifical pairs: a, b is the same as b, a
    df_filtered = df_merged.loc[df_merged.X < df_merged.Y, :].reset_index(drop=True)
    mask = np.array([hpo.exists_path(df_filtered.X[i], df_filtered.Y[i]) or 
                     hpo.exists_path(df_filtered.Y[i], df_filtered.X[i]) for i in np.arange(len(df_filtered))])
    df_filtered = df_filtered.loc[np.logical_not(mask), ].reset_index(drop=True)
    return df_filtered

def entropy(case, control):
    total = case + control 
    h = -(case / total * np.log2(case/total) + control/total * np.log2(control/total))
    return h
    

def load_p_values(path):
    with open(path, 'rb') as f:
        p = pickle.load(f)
    return p

convert_to_percent = np.vectorize(lambda x: ' {:.2f}%'.format(x * 100))

In [46]:
primary_only = True
if primary_only:
    diag_dir = "primary_only"
else:
    diag_dir = "primary_and_secondary"

    
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_textHpo_labHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_textHpo_labHpo = pickle.load(f)
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_textHpo_textHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_textHpo_textHpo = pickle.load(f)
with open('../../../data/mf_regarding_diseases/{}/summaries_diagnosis_labHpo_labHpo.obj'.format(diag_dir), 'rb') as f:
    summaries_diagnosis_labHpo_labHpo = pickle.load(f)
summaries = {('textHpo', 'labHpo'): summaries_diagnosis_textHpo_labHpo,
             ('textHpo', 'textHpo'): summaries_diagnosis_textHpo_textHpo,
            ('labHpo', 'labHpo'): summaries_diagnosis_labHpo_labHpo}

In [47]:
# map column names to more meaningful name for this context
name_dict = {
        'X': 'P1', 
        'Y': 'P2', 
        'mf_Xz': 'mf_P1_diag',
        'mf_Yz': 'mf_P2_diag',
        'mf_XY_z': 'mf_P1P2_diag',
        'mf_XY_given_z': 'mf_P1P2_given_diag',
        'mf_XY_omit_z': 'mf_P1P2_omit_diag',
        'p_mf_XY_z': 'p_mf_P1P2_diag',
        'p_mf_XY_given_z': 'p_mf_P1P2_given_diag',
        'p_mf_XY_omit_z': 'p_mf_P1P2_omit_diag',
        'mf_Xz': 'mf_P1_diag', 
        'mf_Yz': 'mf_P2_diag',  
        'p_mf_Xz': 'p_mf_P1_diag', 
        'p_mf_Yz': 'p_mf_P2_diag'   
    }

### Output rendered csv and html files for selected diseases and phenotype sources

In [49]:
# change the following variables to analyze
disease = '493'
p1_source = 'textHpo'
p2_source = 'labHpo'


# calculate mutual information from summary statistics
summaries_p1_p2 = summaries.get((p1_source, p2_source))
summary_statistics = summaries_p1_p2[disease]
mf_diagnosis_phenotypes = mf.MutualInfoXYz(summary_statistics)

# calculate p value on Helix for disease of interest: input-summary statistics; output-a dictionary of p values
# TODO: get script 

# load p values
p_values_file_name = 'p_value_{}_{}_{}_{}.obj'.format(p1_source, p2_source, disease, diag_dir)
p_values_file_path = os.path.join('../../../data/mf_regarding_diseases/', diag_dir, disease, p_values_file_name)
p_values = load_p_values(p_values_file_path)

df_mf_Xz, df_mf_Yz, df_mf_XY_z = mf_dataframes(mf_diagnosis_phenotypes, **p_values)

df_mf_XY_z_filtered = filter_df(df_mf_Xz, df_mf_Yz, df_mf_XY_z)

# rename column names for this context using the dictionary defined above
df_mf_Xz = df_mf_Xz.rename(columns=name_dict, errors='ignore') 
df_mf_Yz = df_mf_Yz.rename(columns=name_dict, errors='ignore') 
df_mf_XY_z_filtered = df_mf_XY_z_filtered.rename(columns=name_dict, errors='ignore')

# label HPO term ids with their names
df_mf_Xz['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_Xz.P1]
df_mf_Yz['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_Yz.P2]
df_mf_XY_z_filtered['P1_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z_filtered.P1]
df_mf_XY_z_filtered['P2_label'] = [hpo_term_map.get(termid) for termid in df_mf_XY_z_filtered.P2]

# sort by desired columns
df_mf_Xz = df_mf_Xz.sort_values(by='mf_P1_diag', ascending=False).reset_index(drop=True)
df_mf_Yz = df_mf_Yz.sort_values(by='mf_P2_diag', ascending=False).reset_index(drop=True)
df_mf_XY_z_filtered = df_mf_XY_z_filtered.sort_values(by='synergy', ascending=False).reset_index(drop=True)

# output to csv file
# just save df_mf_XY_z_filtered as it contains data in df_mf_Xz and df_mf_Yz
csv_file_name = 'df_synergy_{}_{}_{}.csv'.format(p1_source, p2_source, disease)
csv_file_path = os.path.join('../../../data/mf_regarding_diseases/', diag_dir, csv_file_name)
df_mf_XY_z_filtered.head(n = 20)
df_mf_XY_z_filtered.to_csv(csv_file_path)

# render html


# output cytoscape files
percentile = 0.01
n = math.floor(len(df_mf_XY_z_filtered) * percentile)

df_4_cytoscape = df_mf_XY_z_filtered \
    .assign(P1 = lambda x: 'Rad_' + x['P1']) \
    .assign(P2 = lambda x: 'Lab_' + x['P2']) \
    .head(n = n)


# edges
edges_path = os.path.join('../../../data/mf_regarding_diseases', diag_dir, 'cytoscape', 'edges_{}_{}_{}.csv'.format(p1_source, p2_source, disease))
df_4_cytoscape.loc[:, ['P1', 'P2', 'synergy', 'p_synergy']].to_csv(edges_path)

# nodes
nodes = pd.DataFrame(data={'term_id': np.concatenate([df_4_cytoscape.P1, df_4_cytoscape.P2]), 
                           'term_label': np.concatenate([df_4_cytoscape.P1_label, df_4_cytoscape.P2_label]),
                          'type': np.repeat(['Rad', 'Lab'], len(df_4_cytoscape))}).drop_duplicates()
nodes_path = os.path.join('../../../data/mf_regarding_diseases', diag_dir, 'cytoscape', 'nodes_textHpo_labHpo_{}.csv'.format( disease))
nodes.to_csv(nodes_path)
